In [2]:
# load src/
import sys
import random
sys.path.append('../')
import src.dataset

import numpy as np
from tqdm.auto import tqdm, trange

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
X_train, X_valid, X_test = src.dataset.load_datasets()

In [4]:
# For final submission:
import scipy
X_train = X_train + X_valid
#mean = X_train[X_train.nonzero()].mean()

In [5]:
!nvidia-smi

Tue Jun 23 14:24:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    On   | 00000000:01:00.0 Off |                  N/A |
|  0%   41C    P8    16W / 230W |     42MiB /  8117MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
X_train.count_nonzero(), X_valid.count_nonzero(), X_test.count_nonzero()
valid_indices = list(set(zip(X_train.nonzero()[0], X_train.nonzero()[1])))
# valid_per_line = [[] for _ in range(10000)]
# for i, j in valid_indices:
#     valid_per_line[i].append((i,j))

In [7]:
# from collections import Counter
# def count_vals(X, valid_indices):
#     counts = {}
#     for i, j in valid_indices:
#         val = X[i, j]
#         counts.setdefault(val, 0)
#         counts[val] += 1
#     return counts
# print(count_vals(X_train, valid_indices))

In [8]:
# import random
# X_unbias = X_train.copy()
# for i, j in valid_indices:
#     val = X_train[i, j]
#     if val >= 3 and random.random() < 0.5:
#         X_unbias[i, j] = 0

In [9]:
# valid_indices_unbias = list(set(zip(X_unbias.nonzero()[0], X_unbias.nonzero()[1])))
# print(count_vals(X_unbias, valid_indices))

In [10]:
def _add_regularization(model):
    l2_reg = None
    for W in model.parameters():
        if l2_reg is None:
            l2_reg = W.norm(2)
        else:
            l2_reg = l2_reg + W.norm(2)
    return l2_reg

In [11]:
class Autoencoder(nn.Module):

    def __init__(self):
        super().__init__()
        self.user_dim = 10000
        self.item_dim = 1000
        self.embed_dim = 150
#         self.user_embed = nn.Embedding(self.user_dim, self.embed_dim)
#         self.item_embed = nn.Embedding(self.item_dim, self.embed_dim)
        self.layer_1 = nn.Linear(1000, 100)
        self.layer_1a = nn.Linear(100, 100)
#         self.layer_2 = nn.Linear(500, 500)
        
        self.vae = nn.Linear(100, 100)
        
        self.layer_3 = nn.Linear(50, 100)
        
        self.cls_layer = nn.Linear(100, 1000)
    
    def forward(self, data):
        net_data = data

        # Feedforward layers
        net_data = F.tanh(self.layer_1(net_data))
        net_data = F.dropout(net_data, training=self.training)
        
        net_data = F.relu(self.layer_1a(net_data))
        net_data = F.dropout(net_data, training=self.training)
        
#         net_data = F.tanh(self.layer_2(net_data))
#         net_data = F.dropout(net_data)
        
        vae = self.vae(net_data)
        mus_q, log_sigmas_q = torch.split(vae, 50, dim=-1)
        stds_q = torch.exp(0.5 * log_sigmas_q)
        
        KL = 0.5 * (-log_sigmas_q + torch.exp(log_sigmas_q) + mus_q ** 2 - 1)
        KL = torch.sum(KL, dim=-1)
        KL = torch.mean(KL)
        
        if not self.training:
            sampled_z = mus_q
        else:
            eps = torch.randn(stds_q.shape, dtype=torch.float, device=device)
            sampled_z = mus_q + eps * stds_q
        
#         sampled_z = F.dropout(sampled_z, training=self.training)
        
        net_data = F.tanh(self.layer_3(sampled_z))
        net_data = F.dropout(net_data, training=self.training)
        
        # Predict score for position A_i_j
        y_score = self.cls_layer(net_data)
#         y_score = (4.0 * torch.sigmoid(y_score)) + 1.0
#         return torch.sigmoid(y_score) * 6.0
        return y_score, KL

In [12]:
try:
    del model
except:
    pass
model = Autoencoder().cuda()

In [13]:
device = torch.device('cuda')
opt = torch.optim.Adam(model.parameters(), lr=3e-4)

In [14]:
print(X_train.count_nonzero(), X_valid.count_nonzero())

def get_predictions(A):
    model.eval()
#     model.train()
    A_pred = np.zeros((10000, 1000))
    
    valid_indices = list(set(zip(A.nonzero()[0], A.nonzero()[1])))
    
    for i in range(0, 10000, 64):
        #if i % (64 * 10) == 0:
        #    print(i)
#        X = valid_indices[i:i+64]
        X = X_train[i:i+64].todense()
        #X[X == 0.] = 0.
        X_ = torch.tensor(X, dtype=torch.float, device=device)
        #print(X_)
        
        y_preds, KL = model(X_)
        if i == 0 and False:
            print(X)
            print(model.training)
            print(y_preds)
        
        if USE_CLS:
            y_preds = torch.argmax(torch.softmax(y_preds, dim=-1), dim=-1)
            if i == 0:
                print(y_preds)
        for i_ in range(y_preds.shape[0]):
            x_i = y_preds[i_].detach().cpu()
            A_pred[i+i_] = x_i
            
            #for j_ in range(y_preds.shape[1]):
#                x_ij = y_preds[i_, j_]
#                A_pred[i+i_, j_] = x_ij
        
    #print(A_pred[0])
    ##print("Stats:", np.mean(A_pred[A_pred.nonzero()]), np.std(A_pred.nonzero()))
    return A_pred, valid_indices

def compute_loss():
    A_pred, valid_indices = get_predictions(X_valid)
    losses = np.square(X_valid - A_pred)
    losses = [losses[i,j] for (i, j) in valid_indices]
    mean_loss = np.mean(losses)
    return mean_loss

compute_loss()

1176952 251385


/home/goovi/envs/env/lib/python3.6/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


NameError: name 'USE_CLS' is not defined

In [15]:
train_indices_perline = [[] for i_ in range(10000)]
for i, j in valid_indices:
    train_indices_perline[i].append((i, j))

for line in train_indices_perline:
    assert line
assert len(train_indices_perline) == 10000

In [16]:
mean_for_line = []
for line in X_train:
    mean_for_line.append(line.mean())
assert len(mean_for_line) == 10000
print(mean_for_line[0])
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

0.09300000000000003


In [18]:
loss_avg = []
USE_CLS = False
anneal_alpha = 0.5  # TODO!


for i in range(0, 300000):
    # randomly sample 32 points from training matrix
    model.train()
    #X = random.sample(valid_indices, k=32)
    random_lines = random.sample(range(0, 10000), k=32)
    X = X_train[random_lines].todense()
    #X[X == 0.] = mean_for_line[0]
    
    
    
#     y_true = X
    #y_true = [[X_train[Xi]] for Xi in X]
    X = torch.tensor(X, dtype=torch.float, device=device)
#     y_true = torch.tensor(y_true, dtype=torch.long if USE_CLS else torch.float, device=device)
    # step!
    y_true = X
    
    #########
#     X = torch.tensor([[-1, -1], [0, 0], [2, 2]], dtype=torch.float)
#     y_true = torch.tensor([[-1.], [0.], [2.]], dtype=torch.float)
    #########
    
    opt.zero_grad()
    y_preds, KL = model(X + torch.randn(X.shape, device=device) * 0.5)
    
    valid_mask = torch.zeros_like(y_preds, device=device)
    for i_ in range(X.shape[0]):
        i_real = random_lines[i_]
        indices = [j for _,j in train_indices_perline[i_real]]
        valid_mask[i_, indices] = 1.0
#     y_preds *= valid_mask
    
    if USE_CLS:
        loss_fn = torch.nn.CrossEntropyLoss(
            weight=torch.tensor((0., 3., 3., 1., 1., 1.), device=device))
#         print(y_preds.shape, y_true.shape)
        loss = loss_fn(y_preds, y_true.reshape(-1,))
    else:
        loss = (y_preds - y_true) ** 2
        loss = loss[valid_mask != 0.]
#         print(y_true[valid_mask != 0.])
#         print(loss)
    
    neg_ELBO = loss + anneal_alpha * KL# + 2 * reg_var
    
    # update avg loss
    loss_avg.insert(0, loss.mean().item())
    loss_avg = loss_avg[:1000]
    
    if i % 1000 == 0:
        print("It %d Avg loss: %.4f" % (i, np.mean(loss_avg)))
    if i and i % 1000 == 0:
        mean_loss = compute_loss()
        print("Mean loss: %.4f" % mean_loss)
    
#     loss += _add_regularization(model) * 0.1
    loss.mean().backward()
    #neg_ELBO.mean().backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
    opt.step()


It 0 Avg loss: 0.9394
It 1000 Avg loss: 0.9670
Mean loss: 0.9213
It 2000 Avg loss: 0.9667
Mean loss: 0.9183
It 3000 Avg loss: 0.9665
Mean loss: 0.9190
It 4000 Avg loss: 0.9668
Mean loss: 0.9191
It 5000 Avg loss: 0.9650
Mean loss: 0.9202
It 6000 Avg loss: 0.9641
Mean loss: 0.9196
It 7000 Avg loss: 0.9634
Mean loss: 0.9203
It 8000 Avg loss: 0.9646
Mean loss: 0.9190
It 9000 Avg loss: 0.9649
Mean loss: 0.9198
It 10000 Avg loss: 0.9662
Mean loss: 0.9189
It 11000 Avg loss: 0.9662
Mean loss: 0.9187
It 12000 Avg loss: 0.9642
Mean loss: 0.9192
It 13000 Avg loss: 0.9669
Mean loss: 0.9192
It 14000 Avg loss: 0.9669
Mean loss: 0.9176
It 15000 Avg loss: 0.9635
Mean loss: 0.9187
It 16000 Avg loss: 0.9639
Mean loss: 0.9185
It 17000 Avg loss: 0.9635
Mean loss: 0.9178
It 18000 Avg loss: 0.9662
Mean loss: 0.9187
It 19000 Avg loss: 0.9658
Mean loss: 0.9193
It 20000 Avg loss: 0.9609
Mean loss: 0.9199
It 21000 Avg loss: 0.9616
Mean loss: 0.9180
It 22000 Avg loss: 0.9660
Mean loss: 0.9178
It 23000 Avg loss: 

KeyboardInterrupt: 

In [19]:
mean_loss = compute_loss()
print(mean_loss)
# 1e-3 -> 1e-4 on plateau, as small as possible

0.9158613476608835


In [20]:
# 1. Don't use the classifier, it's not good
# 2. With bs=32, clsweights 22111 adam and 100 neurons it works
# Trained on 1e-4 to 100k, 1.035 valid. loss
# Trained on 1e-5 to 150k, 1.025 valid. loss
# Trained on 1e-5 to 200k, 1.025 valid. loss
# 3. With clsweights 33111 it is the same up to 100k
# 4. With 200 and 300 neurons (150 embedding) the same up to 100k
# 5. With bottleneck of 50 neurons the same up to 100k
## SCORE: 1.05, not good
# Avg loss @100k: 0.95, @200k: 0.92

# With everything, avg loss @300k: 0.87, but worse than before
# Again just 4/5:
# 0.98 and 1.04

In [21]:
import pickle
with open("array-for-svd.pkl", "wb") as wp:
    pickle.dump(get_predictions(X_train)[0], wp)


In [22]:
# Submission

In [23]:
import pandas as pd

def export_and_save(target, preds):
    target_rows, target_cols = target.nonzero()
    ids = [f"r{row+1}_c{col+1}" for row, col in zip(target_rows, target_cols)]
    scores = [preds[row, col] for row, col in zip(target_rows, target_cols)]
    scores = [score if score <= 5.0 else 5.0 for score in scores]
    scores = [score if score >= 1.0 else 1.0 for score in scores]
    df = pd.DataFrame({"Id": ids, "Prediction": scores})
    print("---Please check---")
    print(df.head())
    df.to_csv("preds.csv", index=False)

export_and_save(X_test, get_predictions(X_test)[0])

---Please check---
        Id  Prediction
0    r1_c4    3.602658
1    r1_c8    3.342115
2   r1_c21    3.082139
3  r1_c102    3.803452
4  r1_c127    3.275280


In [24]:
# 3.45
# 3.32
# 2.98
# 3.75
# 3.25
# -> with 1.008 valid. score

In [25]:
# 3.50
# 3.12
# 2.98
# 3.88
# 3.27
# -> with 1.007 valid. score (RMSE 1.00359)
# --> SUBMISSION VAE #2 (RMSE 1.0422) 

In [26]:
import pickle
with open("array-for-svd.pkl", "rb") as fp:
    arr = pickle.load(fp)
    print(arr.shape, arr[0,7])
    del arr

(10000, 1000) 3.3421154022216797


In [ ]:
#---VAE #3---
#        Id  Prediction
#0    r1_c4    3.786128
#1    r1_c8    2.818319
#2   r1_c21    3.745162
#3  r1_c102    4.174467
#4  r1_c127    4.057493
# -> 1.003 valid. score (RMSE 1.0015)
# --> SUBMISSION VAE #3 (1.00239)